In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer , pipeline
import torch

/Users/nima/Projects/persian_poetry generator/venvPPG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
model_path = "../gpt2-farsi-poetry"

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(42001, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=42001, bias=False)
)

In [6]:
def generate_poem(
    model,
    tokenizer,
    prompt="<START>",
    max_length=100,
    num_return_sequences=1,
    temperature=1.0,
    top_k=50,
    top_p=0.95,
    device=None,
    clean_output=True
):
    """
    Generate Persian poems from a fine-tuned GPT-2 model.

    Args:
        model: Trained GPT2LMHeadModel
        tokenizer: Associated tokenizer
        prompt (str): Starting prompt, usually "<START>"
        max_length (int): Maximum length of the generated poem
        num_return_sequences (int): Number of poems to generate
        temperature (float): Sampling temperature (>1 = more randomness)
        top_k (int): Top-K sampling
        top_p (float): Top-P (nucleus) sampling
        device (int or None): Device to use, e.g., 0 for GPU or None for auto
        clean_output (bool): Replace <LINE_BREAK> and <END> in output

    Returns:
        List of generated poems as strings
    """
    if device is None:
        device = 0 if torch.cuda.is_available() else -1

    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        do_sample=True,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.convert_tokens_to_ids("<END>")
    )

    poems = []
    for output in output_sequences:
        text = tokenizer.decode(output, skip_special_tokens=False)

        if clean_output:
            text = text.replace("<START>", "")
            text = text.replace("<END>", "")
            text = text.replace("<LINE_BREAK>", "\n")
            text = text.replace("<PAD>", "")
            text = text.strip()

        poems.append(text)

    return poems if num_return_sequences > 1 else poems[0]


In [7]:
# Robaee
poem = generate_poem(
    model=model,
    tokenizer=tokenizer,
    prompt="<START><STYLE:ROBAEE>",
    max_length=80,
    temperature=0.7,
    top_k=70,
    top_p=0.9
)
print(poem)


<STYLE:ROBAEE>  دلدار چو می به ما میبخشد 
 وز ما به ما هوای ما میبخشد 
 ای دل تو را از ما چه میخواهی؟ 
 ما را به تو جای ما میبخشد <|endoftext|>


In [8]:
# Ghazal
ghazal = generate_poem(
    model=model,
    tokenizer=tokenizer,
    prompt="<START><STYLE:GHAZAL>",
    max_length=120,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)
print(ghazal)

<STYLE:GHAZAL> بیا که تا مرا به تو باز آیم 
 ز درد عشق تو چو جان فزای آیم 
 ز درد عشق تو جانم به لب آمد 
 ز درد عشق تو چون تو جان فزای آیم 
 به هر که روی تو باشد از روی تو 
 به چشم من به تو چون تو به جای آیم 
 نه دل ز دست تو بر باد می رود 
 نه صبر از دست تو بربای آیم


In [74]:
# Masnavi
masnavi = generate_poem(
    model=model,
    tokenizer=tokenizer,
    prompt="<START><STYLE:MASNAVI>",
    max_length=140,
    temperature=0.8,
    top_k=70,
    top_p=0.95
)
print(masnavi)

<STYLE:MASNAVI>  تو ای شاه، مکن دعوی شاهی 
 که پادشاهی طلبد، تو شاهی 
 غلام شاه شدن، اگر چه تو باشی 
 به جرم بندگی، غلام شاه شدن 
 چو شاه، غلام توست، غلام اوست 
 به هر کسی که هست غلام اوست 
 به هیچ صورت، آن چنان نباشد 
 که در همه عالم، آن چنان نباشد 
 به هر زبان که سخن گوید کسی 
 سخن من شنو، به جای سخن
